### Doing
 - [ ] Add CMS_IDs to Outbreaks and Facilities

### To Do
 - [ ] Group Facilities by CMS_Provider_Num WHERE Available
 
### DONE
 - [x] Separate funtions into separate Python files


In [1]:
import pandas as pd
import urllib3 as urllib
import urllib.request as urllib2
import json
import glob
import IPython.display
import altair as alt

from Modules.IL_Functions import *

pd.options.display.max_columns = None

## 1 - Get "Raw Data" from website and store in file

In [2]:
[reporting_date, ltc_data] = pull_IL_json_from_web()

In [3]:
# print('Source File: ' + str(json_file))
[reporting_date, summary, outbreak_df, df_facilities, df_county] = process_IL_dict(ltc_data, ltc_name2cms_id, display_dfs=False)

Date: 2021-02-12
Cases: 74147
Deaths: 9656
Outbreaks: 2029
Open Outbreaks: 904
Closed Outbreaks: 1125
Facilities: 1580


In [4]:
# outbreak_df.groupby(['County', 'FacilityName']).sum()

In [5]:
df_facilities.head(5)

ReportingDate  confirmed_cases  \
County FacilityName2              CMS_ProvNum                                  
Adams  Adams Pointe Senior Living No Match       2021-02-12                4   
       Bickford Cottage           No Match       2021-02-12                9   
       Blessing Hospital SNU      No Match       2021-02-12                6   
       Bradford Villa             No Match       2021-02-12               21   
       Cedarhurst                 No Match       2021-02-12               26   

                                               deaths       CFR  outbreaks  \
County FacilityName2              CMS_ProvNum                                
Adams  Adams Pointe Senior Living No Match          0  0.000000          1   
       Bickford Cottage           No Match          0  0.000000          1   
       Blessing Hospital SNU      No Match          3  0.500000          1   
       Bradford Villa             No Match          3  0.142857          2   
       Cedarhurst                 No Match          1  0.038462          1   

                                               Closed_Outbreaks  \
County FacilityName2              CMS_ProvNum                     
Adams  Adams Pointe Senior Living No Match                    1   
       Bickford Cottage           No Match                    0   
       Blessing Hospital SNU      No Match                    0   
       Bradford Villa             No Match                    1   
       Cedarhurst                 No Match                    1   

                                               Open_Outbreaks  facilities  
County FacilityName2              CMS_ProvNum                              
Adams  Adams Pointe Senior Living No Match                  0           1  
       Bickford Cottage           No Match                  1           1  
       Blessing Hospital SNU      No Match                  1           1  
       Bradford Villa             No Match                  1           1  
       Cedarhurst                 No Match                  0           1

In [6]:
df_facilities.sort_values(by='Open_Outbreaks', ascending=False).head(20)

ReportingDate  \
County    FacilityName2                    CMS_ProvNum                 
Cook      Symphony of Morgan Park          145764         2021-02-12   
          Austin Oasis                     145834         2021-02-12   
          Mosaic of Lakeshore              145244         2021-02-12   
          Symphony of South Shore          145977         2021-02-12   
Lake      Rolling Hills Manor              145443         2021-02-12   
Cook      Arbour Health Care Center        146169         2021-02-12   
          SHERIDAN VILLAGE NURSING HOME    No Match       2021-02-12   
          Aperion Care West Ridge          145832         2021-02-12   
          Alden North Shore                145984         2021-02-12   
Vermilion Accolade Health Care of Danville No Match       2021-02-12   
Cook      Citadel of Skokie                145468         2021-02-12   
          Balmoral Home                    145796         2021-02-12   
          Lakeview Nursing and Rehab       145654         2021-02-12   
Knox      Seminary Manor                   145598         2021-02-12   
LaSalle   Aperion Care Marseilles          145295         2021-02-12   
Knox      Marigold                         No Match       2021-02-12   
          Knox County Nursing Home         145697         2021-02-12   
          Kensington Nursing Home          No Match       2021-02-12   
          Heartland Health                 No Match       2021-02-12   
          Serenity of Galesburg            145987         2021-02-12   

                                                        confirmed_cases  \
County    FacilityName2                    CMS_ProvNum                    
Cook      Symphony of Morgan Park          145764                   214   
          Austin Oasis                     145834                   118   
          Mosaic of Lakeshore              145244                   133   
          Symphony of South Shore          145977                   231   
Lake      Rolling Hills Manor              145443                   125   
Cook      Arbour Health Care Center        146169                    90   
          SHERIDAN VILLAGE NURSING HOME    No Match                  11   
          Aperion Care West Ridge          145832                    74   
          Alden North Shore                145984                    14   
Vermilion Accolade Health Care of Danville No Match                  23   
Cook      Citadel of Skokie                145468                    53   
          Balmoral Home                    145796                    73   
          Lakeview Nursing and Rehab       145654                   100   
Knox      Seminary Manor                   145598                   134   
LaSalle   Aperion Care Marseilles          145295                   103   
Knox      Marigold                         No Match                  45   
          Knox County Nursing Home         145697                   113   
          Kensington Nursing Home          No Match                  11   
          Heartland Health                 No Match                  74   
          Serenity of Galesburg            145987                   104   

                                                        deaths       CFR  \
County    FacilityName2                    CMS_ProvNum                     
Cook      Symphony of Morgan Park          145764           26  0.121495   
          Austin Oasis                     145834           16  0.135593   
          Mosaic of Lakeshore              145244           29  0.218045   
          Symphony of South Shore          145977           36  0.155844   
Lake      Rolling Hills Manor              145443           19  0.152000   
Cook      Arbour Health Care Center        146169            3  0.033333   
          SHERIDAN VILLAGE NURSING HOME    No Match          0  0.000000   
          Aperion Care West Ridge          145832           14  0.189189   
          Alden North Shore                145984          

In [7]:
outbreak_df.head(5)

,reporting_date,County,FacilityName,confirmed_cases,deaths,ReportDate,status,CFR,outbreaks,county-facName,CMS_ProvNum,Closed_Outbreaks,Open_Outbreaks,FacilityName2
0,2021-02-12,Adams,Chaddock,23,0,2021-02-12T00:00:00,Closed,0.000000,1,ADAMS-CHADDOCK,No Match,1,0,Chaddock
1,2021-02-12,Adams,Illinois Veterans Home Quincy,239,24,2021-02-12T00:00:00,Open,0.100418,1,ADAMS-ILLINOIS VETERANS HOME QUINCY,No Match,0,1,Illinois Veterans Home Quincy
2,2021-02-12,Adams,Good Samaritan Home,75,6,2021-02-12T00:00:00,Open,0.080000,1,ADAMS-GOOD SAMARITAN HOME,145773,0,1,Good Samaritan Home
3,2021-02-12,Adams,Adams Pointe Senior Living,4,0,2021-02-12T00:00:00,Closed,0.000000,1,ADAMS-ADAMS POINTE SENIOR LIVING,No Match,1,0,Adams Pointe Senior Living
4,2021-02-12,Adams,Bradford Villa,8,1,2021-02-12T00:00:00,Closed,0.125000,1,ADAMS-BRADFORD VILLA,No Match,1,0,Bradford Villa


In [8]:
df_facilities.head(10)

ReportingDate  \
County FacilityName2                 CMS_ProvNum                 
Adams  Adams Pointe Senior Living    No Match       2021-02-12   
       Bickford Cottage              No Match       2021-02-12   
       Blessing Hospital SNU         No Match       2021-02-12   
       Bradford Villa                No Match       2021-02-12   
       Cedarhurst                    No Match       2021-02-12   
       Chaddock                      No Match       2021-02-12   
       Curtis Creek                  No Match       2021-02-12   
       Golden Good Shepperd Home     No Match       2021-02-12   
       Good Samaritan Home           145773         2021-02-12   
       Illinois Veterans Home Quincy No Match       2021-02-12   

                                                  confirmed_cases  deaths  \
County FacilityName2                 CMS_ProvNum                            
Adams  Adams Pointe Senior Living    No Match                   4       0   
       Bickford Cottage              No Match                   9       0   
       Blessing Hospital SNU         No Match                   6       3   
       Bradford Villa                No Match                  21       3   
       Cedarhurst                    No Match                  26       1   
       Chaddock                      No Match                  23       0   
       Curtis Creek                  No Match                   6       0   
       Golden Good Shepperd Home     No Match                  54       9   
       Good Samaritan Home           145773                    75       6   
       Illinois Veterans Home Quincy No Match                 239      24   

                                                       CFR  outbreaks  \
County FacilityName2                 CMS_ProvNum                        
Adams  Adams Pointe Senior Living    No Match     0.000000          1   
       Bickford Cottage              No Match     0.000000          1   
       Blessing Hospital SNU         No Match     0.500000          1   
       Bradford Villa                No Match     0.142857          2   
       Cedarhurst                    No Match     0.038462          1   
       Chaddock                      No Match     0.000000          1   
       Curtis Creek                  No Match     0.000000          1   
       Golden Good Shepperd Home     No Match     0.166667          1   
       Good Samaritan Home           145773       0.080000          1   
       Illinois Veterans Home Quincy No Match     0.100418          1   

                                                  Closed_Outbreaks  \
County FacilityName2                 CMS_ProvNum                     
Adams  Adams Pointe Senior Living    No Match                    1   
       Bickford Cottage              No Match                    0   
       Blessing Hospital SNU         No Match                    0   
       Bradford Villa                No Match                    1   
       Cedarhurst                    No Match                    1   
       Chaddock                      No Match                    1   
       Curtis Creek                  No Match                    0   
       Golden Good Shepperd Home     No Match                    1   
       Good Samaritan Home           145773                      0   
       Illinois Veterans Home Quincy No Match                    0   

                                                  Open_Outbreaks  facilities  
County FacilityName2                 CMS_ProvNum                              
Adams  Adams Pointe Senior Living    No Match                  0           1  
       Bickford Cottage              No Match                  1           1  
       Blessing Hospital SNU         No Match                  1           1  
       Bradford Villa                No Match                  1           1  
       Cedarhurst                    No Match                  0           1  
       Chaddock                      No Match               

In [9]:
# # display(df_facilities.reset_index().head(5))
# df_facilities.head(5)

In [10]:
df_facilities.reset_index().groupby(['CMS_ProvNum']).count().sort_values(by='County', ascending=False)

,County,FacilityName2,ReportingDate,confirmed_cases,deaths,CFR,outbreaks,Closed_Outbreaks,Open_Outbreaks,facilities
CMS_ProvNum,,,,,,,,,,
No Match,1036,1036,1036,1036,1036,1036,1036,1036,1036,1036
145364,3,3,3,3,3,3,3,3,3,3
14E888,2,2,2,2,2,2,2,2,2,2
145819,2,2,2,2,2,2,2,2,2,2
145411,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...
145639,1,1,1,1,1,1,1,1,1,1
145638,1,1,1,1,1,1,1,1,1,1
145637,1,1,1,1,1,1,1,1,1,1


In [11]:
df_facilities = df_facilities.reset_index()
s = df_facilities.CMS_ProvNum.value_counts()
dupNamesList = list(s.where((s > 1) & (s < 100)).dropna().keys())

In [12]:
s = df_facilities.CMS_ProvNum.value_counts()
dupNamesList = list(s.where((s > 1) & (s < 100)).dropna().keys())

In [13]:
dupFacNames_df = df_facilities[df_facilities['CMS_ProvNum'].isin(dupNamesList)].sort_values(by='CMS_ProvNum')
dupFacNames_df.to_csv('IL_dup_fac_names.csv')
dupFacNames_df

,County,FacilityName2,CMS_ProvNum,ReportingDate,confirmed_cases,deaths,CFR,outbreaks,Closed_Outbreaks,Open_Outbreaks,facilities
807,Kane,The Grove Fox Valley,145006,2021-02-12,174,24,0.137931,2,1,1,1
809,Kane,The Grove of Fox Valley,145006,2021-02-12,7,0,0.000000,1,1,0,1
819,Kankakee,Citadel Care Center of Kankakee,145043,2021-02-12,3,0,0.000000,1,1,0,1
820,Kankakee,Citadel of Kankakee,145043,2021-02-12,84,2,0.023810,1,0,1,1
1095,McHenry,Cross Roads Care Center,145222,2021-02-12,71,13,0.183099,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
340,Cook,Little Sisters of the Poor/St. Marys Home,146185,2021-02-12,61,13,0.213115,1,1,0,1
751,Kane,Alden Estates-Courts of Huntley,146186,2021-02-12,54,3,0.055556,1,0,1,1
750,Kane,Alden Estates Courts of Huntley,146186,2021-02-12,9,0,0.000000,1,1,0,1
1217,Peoria,Sharon Healthcare Willows,14E888,2021-02-12,141,8,0.056738,1,0,1,1


# Convert Facility Sheet to CTP format

In [14]:
CTP_cols = ['ReportingDate', 'State', 'County', 
            'City',
            'FacilityName', 'TYPE',
            'CTP Facility Categorization',
            'State/Fed Regulated',
            'State Facility ID',
            'CMS Facility ID',
            'Date outbreak opened', 'Date outreak closed', 'Outbreak Status','Resident Census',
            
'TOTAL RESIDENT CASES', # 'Resident Positives ',
          'TOTAL Resident Probable Positives',
'TOTAL RESIDENT DEATHS', # 'Resident Deaths',
         'TOTAL Resident Probable Deaths',
'TOTAL STAFF CASES', #'Staff Positive',
         'TOTAL Staff Probable Positives',
'TOTAL STAFF DEATHS', #'TOTAL Staff Deaths',
         'TOTAL Staff Probable Deaths',
 'confirmed_cases',
 'TOTAL Resident/Staff Probable Positives',
 'deaths',
 'TOTAL Resident/Staff Probable Deaths',
 
'NEW RESIDENT CASES', #            'Resident Positives',
     'NEW Resident Probable Positives',
 'NEW RESIDENT DEATHS',
     'NEW Resident Probable Deaths',
 'NEW STAFF CASES',
     'NEW Staff Probable Positives',
 'NEW STAFF DEATHS',
     'NEW Staff Probable Deaths',
 'NEW Resident/Staff Positives',
 'NEW Resident/Staff Probable Positives',
 'NEW Resident/Staff Deaths',
 'NEW Resident/Staff Probable Deaths',
 'Residents Tested',
 'Staff Tested',
 'Personal Protective Equipment',
           'COMPLIANCE']

CTP_Blank_Cols = ['City',
             'TYPE',
            'CTP Facility Categorization',
            'State/Fed Regulated',
            'State Facility ID',
            'CMS Facility ID',
            'Date outbreak opened', 'Date outreak closed', 'Resident Census',
            
'TOTAL RESIDENT CASES', # 'Resident Positives ',
          'TOTAL Resident Probable Positives',
'TOTAL RESIDENT DEATHS', # 'Resident Deaths',
         'TOTAL Resident Probable Deaths',
'TOTAL STAFF CASES', #'Staff Positive',
         'TOTAL Staff Probable Positives',
'TOTAL STAFF DEATHS', #'TOTAL Staff Deaths',
         'TOTAL Staff Probable Deaths',

 'TOTAL Resident/Staff Probable Positives',

 'TOTAL Resident/Staff Probable Deaths',
 
'NEW RESIDENT CASES', #            'Resident Positives',
     'NEW Resident Probable Positives',
 'NEW RESIDENT DEATHS',
     'NEW Resident Probable Deaths',
 'NEW STAFF CASES',
     'NEW Staff Probable Positives',
 'NEW STAFF DEATHS',
     'NEW Staff Probable Deaths',
 'NEW Resident/Staff Positives',
 'NEW Resident/Staff Probable Positives',
 'NEW Resident/Staff Deaths',
 'NEW Resident/Staff Probable Deaths',
 'Residents Tested',
 'Staff Tested',
 'Personal Protective Equipment',
           'COMPLIANCE']



In [15]:
def open_closed(outbreaks):
    if outbreaks == 0:
        return 'Closed'
    else:
        return 'Open'
        
for col in CTP_Blank_Cols:
    df_facilities[col] = ''
df_facilities['Outbreak Status'] = df_facilities['Open_Outbreaks'].apply(open_closed) #TO FINISH
df_facilities['State'] = 'IL'
df_facilities['ReportingDate'] = reporting_date.replace('-', '')

In [16]:
df_facilities.rename(columns={'FacilityName2': 'FacilityName'}, inplace=True)
df_facilities[CTP_cols].to_csv('Reporting_data/IL_' + reporting_date + '_Facilities_LTC_data_vCTP.csv', index=False)

In [17]:
df_facilities.head()

,County,FacilityName,CMS_ProvNum,ReportingDate,confirmed_cases,deaths,CFR,outbreaks,Closed_Outbreaks,Open_Outbreaks,facilities,City,TYPE,CTP Facility Categorization,State/Fed Regulated,State Facility ID,CMS Facility ID,Date outbreak opened,Date outreak closed,Resident Census,TOTAL RESIDENT CASES,TOTAL Resident Probable Positives,TOTAL RESIDENT DEATHS,TOTAL Resident Probable Deaths,TOTAL STAFF CASES,TOTAL Staff Probable Positives,TOTAL STAFF DEATHS,TOTAL Staff Probable Deaths,TOTAL Resident/Staff Probable Positives,TOTAL Resident/Staff Probable Deaths,NEW RESIDENT CASES,NEW Resident Probable Positives,NEW RESIDENT DEATHS,NEW Resident Probable Deaths,NEW STAFF CASES,NEW Staff Probable Positives,NEW STAFF DEATHS,NEW Staff Probable Deaths,NEW Resident/Staff Positives,NEW Resident/Staff Probable Positives,NEW Resident/Staff Deaths,NEW Resident/Staff Probable Deaths,Residents Tested,Staff Tested,Personal Protective Equipment,COMPLIANCE,Outbreak Status,State
0,Adams,Adams Pointe Senior Living,No Match,20210212,4,0,0.000000,1,1,0,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Closed,IL
1,Adams,Bickford Cottage,No Match,20210212,9,0,0.000000,1,0,1,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Open,IL
2,Adams,Blessing Hospital SNU,No Match,20210212,6,3,0.500000,1,0,1,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Open,IL
3,Adams,Bradford Villa,No Match,20210212,21,3,0.142857,2,1,1,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Open,IL
4,Adams,Cedarhurst,No Match,20210212,26,1,0.038462,1,1,0,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Closed,IL


# Play Area

In [18]:
df = pd.DataFrame(ltc_data['LTC_Historical_Reported_Cases'])
df.sort_values(by=['reported_date'], inplace=True)
df['new_deaths']= df.deaths.diff()
df['new_confirmed_cases'] = df.confirmed_cases.diff()

In [29]:
df.tail(5)

,confirmed_cases,deaths,reported_date,new_deaths,new_confirmed_cases,legend
4,69398,8986,2021-01-15T00:00:00,316.0,1991.0,LTC
3,71103,9263,2021-01-22T00:00:00,277.0,1705.0,LTC
2,71946,9461,2021-01-29T00:00:00,198.0,843.0,LTC
1,73286,9571,2021-02-05T00:00:00,110.0,1340.0,LTC
0,74147,9656,2021-02-12T00:00:00,85.0,861.0,LTC


In [20]:
chart_title = 'IL LTC Cases by Week'
chart1 = alt.Chart(df).mark_bar().encode(
                     x=alt.X('yearmonthdate(reported_date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='new_confirmed_cases',
                     ).properties( width=400, height=400, title=chart_title)
chart_title = 'IL LTC Deaths by Week'
chart2 = alt.Chart(df).mark_bar().encode(
                     x=alt.X('yearmonthdate(reported_date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='new_deaths',
                     ).properties( width=400, height=400, title=chart_title)
chart1 | chart2

alt.HConcatChart(...)

In [21]:
il_data_df = pd.read_json('https://api.covidtracking.com/v1/states/il/daily.json')
il_data_df['date']= pd.to_datetime(il_data_df['date'].astype(str), format='%Y%m%d')

In [22]:
chart_title = 'IL Cases by Week'
il_data_df['legend'] = 'state'
il_data_df['positiveIncrease_7day'] = il_data_df['positiveIncrease'].rolling(7).sum().round(1)
chart1 = alt.Chart(il_data_df).mark_line(color="#FFAA00").encode(
                     x=alt.X('yearmonthdate(date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='positiveIncrease_7day',
                     color=alt.Color('legend', legend=alt.Legend(title="Cases"))
                     ).properties( width=400, height=400, title=chart_title)

chart_title = 'IL Cases by Week'
df['legend'] = 'LTC'

chart2 = alt.Chart(df).mark_line().encode(
                     x=alt.X('yearmonthdate(reported_date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='new_confirmed_cases',
                     color=alt.Color('legend', legend=alt.Legend(title="Cases"))
                     ).properties( width=400, height=400, title=chart_title)

chart_title = 'IL Deaths by Week'
il_data_df['legend'] = 'state'
il_data_df['deathIncrease_7day'] = il_data_df['deathIncrease'].rolling(7).sum().round(1)
#il_data_df['deathIncrease_7day'] = il_data_df['deathIncrease_7day']  * 7
chart3 = alt.Chart(il_data_df).mark_line(color="#FFAA00").encode(
                     x=alt.X('yearmonthdate(date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='deathIncrease_7day',
                     color=alt.Color('legend', legend=alt.Legend(title="Cases"))
                     ).properties( width=400, height=400, title=chart_title)

chart_title = 'IL Deaths by Week'
df['legend'] = 'LTC'

chart4 = alt.Chart(df).mark_line().encode(
                     x=alt.X('yearmonthdate(reported_date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='new_deaths',
                     color=alt.Color('legend', legend=alt.Legend(title="Cases"))
                     ).properties( width=400, height=400, title=chart_title)
(chart1 + chart2) | (chart3 + chart4)

alt.HConcatChart(...)

In [27]:
#il_data_df.columns

In [24]:
il_data_df.sort_values(by='date', ascending=False)[['date', 'deathIncrease']].head(7)

,date,deathIncrease
0,2021-02-17,25
1,2021-02-16,33
2,2021-02-15,45
3,2021-02-14,34
4,2021-02-13,60
5,2021-02-12,42
6,2021-02-11,116


In [25]:
df.sort_values(by='reported_date', ascending=False)

,confirmed_cases,deaths,reported_date,new_deaths,new_confirmed_cases,legend
0,74147,9656,2021-02-12T00:00:00,85.0,861.0,LTC
1,73286,9571,2021-02-05T00:00:00,110.0,1340.0,LTC
2,71946,9461,2021-01-29T00:00:00,198.0,843.0,LTC
3,71103,9263,2021-01-22T00:00:00,277.0,1705.0,LTC
4,69398,8986,2021-01-15T00:00:00,316.0,1991.0,LTC
5,67407,8670,2021-01-08T00:00:00,373.0,2329.0,LTC
6,65078,8297,2021-01-01T00:00:00,387.0,2478.0,LTC
7,62600,7910,2020-12-25T00:00:00,351.0,2630.0,LTC
8,59970,7559,2020-12-18T00:00:00,605.0,5063.0,LTC
9,54907,6954,2020-12-11T00:00:00,428.0,4489.0,LTC


In [28]:
il_data_df

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,dateChecked,totalTestsViral,positiveTestsViral,negativeTestsViral,positiveCasesViral,deathConfirmed,deathProbable,totalTestEncountersViral,totalTestsPeopleViral,totalTestsAntibody,positiveTestsAntibody,negativeTestsAntibody,totalTestsPeopleAntibody,positiveTestsPeopleAntibody,negativeTestsPeopleAntibody,totalTestsPeopleAntigen,positiveTestsPeopleAntigen,totalTestsAntigen,positiveTestsAntigen,fips,positiveIncrease,negativeIncrease,total,totalTestResultsIncrease,posNeg,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade,legend,positiveIncrease_7day,deathIncrease_7day
0,2021-02-17,IL,1166717,NaN,16154097,NaN,posNeg,17320814,1719.0,NaN,375.0,NaN,176.0,NaN,NaN,2/17/2021 00:00,2021-02-17T00:00:00Z,02/16 19:00,22224.0,NaN,2021-02-17T00:00:00Z,17320814.0,NaN,NaN,NaN,20057.0,2167.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,1795,48142,17320814,49937,17320814,NaN,25,0,38992e483d3f21ec47007e833f92675e0b3a9da2,0,0,0,0,0,,state,NaN,NaN
1,2021-02-16,IL,1164922,NaN,16105955,NaN,posNeg,17270877,1726.0,NaN,385.0,NaN,179.0,NaN,NaN,2/16/2021 00:00,2021-02-16T00:00:00Z,02/15 19:00,22199.0,NaN,2021-02-16T00:00:00Z,17270877.0,NaN,NaN,NaN,20034.0,2165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,1348,45282,17270877,46630,17270877,NaN,33,0,0aa29b4bf0d64cc7bcaddc4d7d685328f0b190d1,0,0,0,0,0,,state,NaN,NaN
2,2021-02-15,IL,1163574,NaN,16060673,NaN,posNeg,17224247,1789.0,NaN,389.0,NaN,184.0,NaN,NaN,2/15/2021 00:00,2021-02-15T00:00:00Z,02/14 19:00,22166.0,NaN,2021-02-15T00:00:00Z,17224247.0,NaN,NaN,NaN,20002.0,2164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,1420,50969,17224247,52389,17224247,NaN,45,0,5d601a2585de201fdab3d3540b9f25313ff3fa42,0,0,0,0,0,,state,NaN,NaN
3,2021-02-14,IL,1162154,NaN,16009704,NaN,posNeg,17171858,1777.0,NaN,373.0,NaN,189.0,NaN,NaN,2/14/2021 00:00,2021-02-14T00:00:00Z,02/13 19:00,22121.0,NaN,2021-02-14T00:00:00Z,17171858.0,NaN,NaN,NaN,19961.0,2160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,1631,63318,17171858,64949,17171858,NaN,34,0,cc7556d41c6a50f2bbad98752964e571e0eb0a83,0,0,0,0,0,,state,NaN,NaN
4,2021-02-13,IL,1160523,NaN,15946386,NaN,posNeg,17106909,1892.0,NaN,425.0,NaN,202.0,NaN,NaN,2/13/2021 00:00,2021-02-13T00:00:00Z,02/12 19:00,22087.0,NaN,2021-02-13T00:00:00Z,17106909.0,NaN,NaN,NaN,19926.0,2161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,2092,82898,17106909,84990,17106909,NaN,60,0,6f283b43c57ac80987d318d92d51c2dab83b05e2,0,0,0,0,0,,state,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2020-03-08,IL,6,NaN,191,44.0,posNeg,197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,0,0,241,0,197,NaN,0,0,cb90d62eebfe7a9be39760212460759db30af22b,0,0,0,0,0,,state,40.0,0.0
347,2020-03-07,IL,6,NaN,191,44.0,posNeg,197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,1,11,241,12,197,NaN,0,0,6402fcfcd364e9119891f9fabd1ae5ad17ea6f02,0,0,0,0,0,,state,27.0,0.0
348,2020-03-06,IL,5,NaN,180,35.0,posNeg,185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,0,15,220,15,185,NaN,0,0,1bafdd16fb468dde9df22b8d40fc6adc5f623e47,0,0,0,0,0,,state,20.0,0.0
349,2020-03-05,IL,5,NaN,165,27.0,posNeg,170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N